## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-11-16-38-56 +0000,nyt,$100 Billion to Revive Venezuela’s Energy Indu...,https://www.nytimes.com/2026/01/10/business/de...
1,2026-01-11-16-33-10 +0000,bbc,"Trump tells Cuba to 'make a deal, before it is...",https://www.bbc.com/news/articles/cx2kv2gn62vo...
2,2026-01-11-16-29-40 +0000,cbc,Trump says no more Venezuelan oil or money to ...,https://www.cbc.ca/news/world/trump-says-no-mo...
3,2026-01-11-16-28-00 +0000,wsj,South Africa Risks Angering Trump by Hosting N...,https://www.wsj.com/world/africa/south-africa-...
4,2026-01-11-16-22-57 +0000,nyt,Iran’s President Warns Protesters Not to Desta...,https://www.nytimes.com/2026/01/11/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,31
77,ice,19
27,iran,18
2,venezuela,13
76,minneapolis,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
171,2026-01-10-17-56-09 +0000,nypost,Trump reposts Lindsey Graham’s tweet warning I...,https://nypost.com/2026/01/10/world-news/trump...,69
138,2026-01-10-21-44-31 +0000,nypost,"New Jersey’s Petty’s Island, now owned by Vene...",https://nypost.com/2026/01/10/us-news/new-jers...,65
162,2026-01-10-19-06-36 +0000,nypost,Trump admin weighing preliminary attack plans ...,https://nypost.com/2026/01/10/world-news/trump...,65
156,2026-01-10-20-08-29 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,64
24,2026-01-11-14-24-00 +0000,wsj,Iran Threatens to Hit U.S. Bases if Trump Stri...,https://www.wsj.com/world/middle-east/iran-war...,64


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
171,69,2026-01-10-17-56-09 +0000,nypost,Trump reposts Lindsey Graham’s tweet warning I...,https://nypost.com/2026/01/10/world-news/trump...
87,54,2026-01-11-04-16-32 +0000,nyt,Anti-ICE Protests Spread Nationwide After Minn...,https://www.nytimes.com/2026/01/10/us/ice-shoo...
138,34,2026-01-10-21-44-31 +0000,nypost,"New Jersey’s Petty’s Island, now owned by Vene...",https://nypost.com/2026/01/10/us-news/new-jers...
82,34,2026-01-11-07-27-11 +0000,nypost,Iran warns US troops and Israel will be target...,https://nypost.com/2026/01/11/world-news/iran-...
181,33,2026-01-10-17-00-00 +0000,wsj,In an industry town that has grown increasingl...,https://www.wsj.com/politics/policy/save-calif...
148,32,2026-01-10-20-53-00 +0000,wsj,The Defense Department has selected Owen West ...,https://www.wsj.com/politics/national-security...
72,27,2026-01-11-10-00-00 +0000,wsj,President Trump’s proposals for the housing ma...,https://www.wsj.com/economy/housing/trumps-new...
177,26,2026-01-10-17-23-27 +0000,nypost,US used powerful mystery weapon that brought V...,https://nypost.com/2026/01/10/world-news/us-us...
94,25,2026-01-11-03-13-45 +0000,missionlocal,‘We’re here to create something positive’: Pop...,https://missionlocal.org/2026/01/were-here-to-...
61,24,2026-01-11-10-14-36 +0000,nypost,Neighbor of Illinois surgeon accused of killin...,https://nypost.com/2026/01/11/us-news/neighbor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
